In [1]:
import pandas as pd
import ast

df = pd.read_csv('output/dataset.csv')
df['sentence'] = df['sentence'].apply(lambda x: ast.literal_eval(x)) # str to dict
df['sentence'] = df['sentence'].apply(lambda x: {**x, 'it': ast.literal_eval(x['it'])} if isinstance(x['it'], str) else x) # there were problem with it dict
df.head()

,name,demonym,sentence
0,Afghanistan,Afghan,"{'eng': {'m': None, 'f': None, 'n': 'I am Afgh..."
1,Angola,Angolan,"{'eng': {'m': None, 'f': None, 'n': 'I am Ango..."
2,Albania,Albanian,"{'eng': {'m': None, 'f': None, 'n': 'I am Alba..."
3,Andorra,Andorran,"{'eng': {'m': None, 'f': None, 'n': 'I am Ando..."
4,United Arab Emirates,Emirati,"{'eng': {'m': None, 'f': None, 'n': 'I am Emir..."


In [2]:
from openai import OpenAI

client = OpenAI(
  organization='org-0M5WYRn7SjJnnKLbFybd5QUE'
)

In [3]:
df.iloc[0]['sentence']

{'eng': {'m': None, 'f': None, 'n': 'I am Afghan.'},
 'fra': {'m': 'Je suis Afghan.', 'f': 'Je suis Afghane.', 'n': None},
 'spa': {'m': 'Soy afgano.', 'f': 'Soy afgana.', 'n': None},
 'deu': {'m': 'Ich bin Afghane.', 'f': 'Ich bin  Afghanin.', 'n': None},
 'it': {'m': 'Sono afghano.', 'f': 'Sono afghana.', 'n': None}}

In [4]:
source_sentence = [x.get('eng', {}).get('n') for x in df['sentence']]
source_sentence[:10]

['I am Afghan.',
 'I am Angolan.',
 'I am Albanian.',
 'I am Andorran.',
 'I am Emirati.',
 'I am Argentine.',
 'I am Armenian.',
 'I am Antiguan.',
 'I am Australian.',
 'I am Austrian.']

## Manual Test1 (example prompt/4o)

In [6]:
from openai import OpenAI
client = OpenAI()

response = client.chat.completions.create(
  model="gpt-4o",
  messages=[
    {
      "role": "system",
      "content": "You will be provided with a sentence in English, and your task is to translate it into French."
    },
    {
      "role": "user",
      "content": "I am Afghan."
    }
  ],
  temperature=0.2,
  max_tokens=256,
  top_p=1
)

response.choices[0].message.content

'Je suis Afghan.'

In [34]:
from openai import OpenAI
client = OpenAI()

response = client.chat.completions.create(
  model="gpt-4o",
  messages=[
    {
      "role": "system",
      "content": "You will be provided with a sentence in English, and your task is to translate it into Spanish."
    },
    {
      "role": "user",
      "content": "I am Afghan."
    }
  ],
  temperature=0.2,
  max_tokens=256,
  top_p=1
)

response.choices[0].message.content

'Soy afgano.'

In [35]:
from openai import OpenAI
client = OpenAI()

response = client.chat.completions.create(
  model="gpt-4o",
  messages=[
    {
      "role": "system",
      "content": "You will be provided with a sentence in English, and your task is to translate it into Italian."
    },
    {
      "role": "user",
      "content": "I am Afghan."
    }
  ],
  temperature=0.2,
  max_tokens=256,
  top_p=1
)

response.choices[0].message.content

'Sono afgano.'

In [36]:
from openai import OpenAI
client = OpenAI()

response = client.chat.completions.create(
  model="gpt-4o",
  messages=[
    {
      "role": "system",
      "content": "You will be provided with a sentence in English, and your task is to translate it into German."
    },
    {
      "role": "user",
      "content": "I am Afghan."
    }
  ],
  temperature=0.2,
  max_tokens=256,
  top_p=1
)

response.choices[0].message.content

'Ich bin Afghane.'

**Result**: gpt-4o does not provide female variant for the above prompt

## Manual Test2 (paper prompt:4o)

In [14]:
response = client.chat.completions.create(
  model="gpt-4o",
  messages=[
    {
      "role": "system",
      "content": "You are a helpful assistant"
    },
    {
      "role": "user",
      "content": "Can you translate the following sentence into French: I am Afghan."
    }
  ],
  temperature=0.2,
  max_tokens=100,
  top_p=1
)

response.choices[0].message.content

'The translation of "I am Afghan" into French is "Je suis Afghan" if you are male, or "Je suis Afghane" if you are female.'

In [20]:
response = client.chat.completions.create(
  model="gpt-4o",
  messages=[
    {
      "role": "system",
      "content": "You are a helpful assistant"
    },
    {
      "role": "user",
      "content": "Can you translate the following sentence into Spanish: I am Afghan."
    }
  ],
  temperature=0.2,
  max_tokens=100,
  top_p=1
)

response.choices[0].message.content

'The translation of "I am Afghan" into Spanish is "Soy afgano" if you are male, or "Soy afgana" if you are female.'

In [21]:
response = client.chat.completions.create(
  model="gpt-4o",
  messages=[
    {
      "role": "system",
      "content": "You are a helpful assistant"
    },
    {
      "role": "user",
      "content": "Can you translate the following sentence into Italian: I am Afghan."
    }
  ],
  temperature=0.2,
  max_tokens=100,
  top_p=1
)

response.choices[0].message.content

'Certainly! The sentence "I am Afghan" can be translated into Italian as "Sono afghano" if you are male, or "Sono afghana" if you are female.'

In [22]:
response = client.chat.completions.create(
  model="gpt-4o",
  messages=[
    {
      "role": "system",
      "content": "You are a helpful assistant"
    },
    {
      "role": "user",
      "content": "Can you translate the following sentence into German: I am Afghan."
    }
  ],
  temperature=0.2,
  max_tokens=100,
  top_p=1
)

response.choices[0].message.content

'The translation of "I am Afghan" into German is "Ich bin Afghane" for a male or "Ich bin Afghanin" for a female.'

**Result**: gpt-4o provides both masculine and feminine variant after changing the prompt

## Manual Test3 (paper prompt: 3.5-turbo)

In [15]:
response = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[
    {
      "role": "system",
      "content": "You are a helpful assistant"
    },
    {
      "role": "user",
      "content": "Can you translate the following sentence into French: I am Afghan."
    }
  ],
  temperature=0.2,
  max_tokens=100,
  top_p=1
)

response.choices[0].message.content

'Certainly! The translation of "I am Afghan" into French is "Je suis Afghan."'

In [23]:
response = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[
    {
      "role": "system",
      "content": "You are a helpful assistant"
    },
    {
      "role": "user",
      "content": "Can you translate the following sentence into Spanish: I am Afghan."
    }
  ],
  temperature=0.2,
  max_tokens=100,
  top_p=1
)

response.choices[0].message.content

'Sure! The translation of "I am Afghan" into Spanish is "Soy afgano/a".'

In [24]:
response = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[
    {
      "role": "system",
      "content": "You are a helpful assistant"
    },
    {
      "role": "user",
      "content": "Can you translate the following sentence into Italian: I am Afghan."
    }
  ],
  temperature=0.2,
  max_tokens=100,
  top_p=1
)

response.choices[0].message.content

'Sure! The translation of "I am Afghan" into Italian is "Sono afghano/a" depending on the gender of the speaker.'

In [25]:
response = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[
    {
      "role": "system",
      "content": "You are a helpful assistant"
    },
    {
      "role": "user",
      "content": "Can you translate the following sentence into German: I am Afghan."
    }
  ],
  temperature=0.2,
  max_tokens=100,
  top_p=1
)

response.choices[0].message.content

'Sure! The translation of "I am Afghan" into German is "Ich bin Afghane."'

**Result**: gpt-3.5 provides different forms depending on target language. <br>
**Hypothesis**: Because Spanish and Italian are high resourced languages

## Manual Test4 (paper prompt: 4)

In [18]:
response = client.chat.completions.create(
  model="gpt-4", 
  messages=[
    {
      "role": "system",
      "content": "You are a helpful assistant"
    },
    {
      "role": "user",
      "content": "Can you translate the following sentence into French: I am Afghan."
    }
  ],
  temperature=0.2,
  max_tokens=100,
  top_p=1
)

response.choices[0].message.content

'Je suis Afghan.'

In [26]:
response = client.chat.completions.create(
  model="gpt-4", 
  messages=[
    {
      "role": "system",
      "content": "You are a helpful assistant"
    },
    {
      "role": "user",
      "content": "Can you translate the following sentence into Spanish: I am Afghan."
    }
  ],
  temperature=0.2,
  max_tokens=100,
  top_p=1
)

response.choices[0].message.content

'Soy afgano.'

In [31]:
response = client.chat.completions.create(
  model="gpt-4", 
  messages=[
    {
      "role": "system",
      "content": "You are a helpful assistant"
    },
    {
      "role": "user",
      "content": "Can you translate the following sentence into Italian: I am Afghan."
    }
  ],
  temperature=0.2,
  max_tokens=100,
  top_p=1
)

response.choices[0].message.content

'Sono afghano.'

In [33]:
response = client.chat.completions.create(
  model="gpt-4", 
  messages=[
    {
      "role": "system",
      "content": "You are a helpful assistant"
    },
    {
      "role": "user",
      "content": "Can you translate the following sentence into German: I am Afghan."
    }
  ],
  temperature=0.2,
  max_tokens=100,
  top_p=1
)

response.choices[0].message.content

'Ich bin Afghane.'

**Result**: gpt-4 provides only masculine form of the sentence

## Manual Test5 (paper prompt: 4-turbo)

In [37]:
response = client.chat.completions.create(
  model="gpt-4-turbo", 
  messages=[
    {
      "role": "system",
      "content": "You are a helpful assistant"
    },
    {
      "role": "user",
      "content": "Can you translate the following sentence into French: I am Afghan."
    }
  ],
  temperature=0.2,
  max_tokens=100,
  top_p=1
)

response.choices[0].message.content

'Je suis Afghan.'

In [38]:
response = client.chat.completions.create(
  model="gpt-4-turbo", 
  messages=[
    {
      "role": "system",
      "content": "You are a helpful assistant"
    },
    {
      "role": "user",
      "content": "Can you translate the following sentence into Spanish: I am Afghan."
    }
  ],
  temperature=0.2,
  max_tokens=100,
  top_p=1
)

response.choices[0].message.content

'Soy afgano.'

In [39]:
response = client.chat.completions.create(
  model="gpt-4-turbo", 
  messages=[
    {
      "role": "system",
      "content": "You are a helpful assistant"
    },
    {
      "role": "user",
      "content": "Can you translate the following sentence into Italian: I am Afghan."
    }
  ],
  temperature=0.2,
  max_tokens=100,
  top_p=1
)

response.choices[0].message.content

'Sure, the translation of "I am Afghan" into Italian is "Sono afghano" if you are male, or "Sono afghana" if you are female.'

In [40]:
response = client.chat.completions.create(
  model="gpt-4-turbo", 
  messages=[
    {
      "role": "system",
      "content": "You are a helpful assistant"
    },
    {
      "role": "user",
      "content": "Can you translate the following sentence into German: I am Afghan."
    }
  ],
  temperature=0.2,
  max_tokens=100,
  top_p=1
)

response.choices[0].message.content

'Ich bin Afghane. (for a male)  \nIch bin Afghanin. (for a female)'

**Result**: Depending on the target language the model gpt-4-turbo provide both variants or only masculine

## Manual Test6 (paper prompt: 4o-mini)

In [43]:
response = client.chat.completions.create(
  model="gpt-4o-mini", 
  messages=[
    {
      "role": "system",
      "content": "You are a helpful assistant"
    },
    {
      "role": "user",
      "content": "Can you translate the following sentence into French: I am Afghan."
    }
  ],
  temperature=0.2,
  max_tokens=100,
  top_p=1
)

response.choices[0].message.content

'The translation of "I am Afghan" into French is "Je suis afghan."'

In [44]:
response = client.chat.completions.create(
  model="gpt-4o-mini", 
  messages=[
    {
      "role": "system",
      "content": "You are a helpful assistant"
    },
    {
      "role": "user",
      "content": "Can you translate the following sentence into Spanish: I am Afghan."
    }
  ],
  temperature=0.2,
  max_tokens=100,
  top_p=1
)

response.choices[0].message.content

'The translation of "I am Afghan" into Spanish is "Soy afgano" (if you are male) or "Soy afgana" (if you are female).'

In [45]:
response = client.chat.completions.create(
  model="gpt-4o-mini", 
  messages=[
    {
      "role": "system",
      "content": "You are a helpful assistant"
    },
    {
      "role": "user",
      "content": "Can you translate the following sentence into Italian: I am Afghan."
    }
  ],
  temperature=0.2,
  max_tokens=100,
  top_p=1
)

response.choices[0].message.content

'The translation of "I am Afghan" into Italian is "Sono afghano" if you are male, or "Sono afghana" if you are female.'

In [46]:
response = client.chat.completions.create(
  model="gpt-4o-mini", 
  messages=[
    {
      "role": "system",
      "content": "You are a helpful assistant"
    },
    {
      "role": "user",
      "content": "Can you translate the following sentence into German: I am Afghan."
    }
  ],
  temperature=0.2,
  max_tokens=100,
  top_p=1
)

response.choices[0].message.content

'The translation of "I am Afghan" into German is "Ich bin Afghane" (if you are male) or "Ich bin Afghanin" (if you are female).'

| **Model**       | **Output Variants**                                                                 |
|------------------|------------------------------------------------------------------------------------|
| **GPT-4o**       | Provides BOTH masculine and feminine variants after changing the prompt.           |
| **GPT-3.5**      | DEPENDING on the target language, provides both variants or only the masculine one.|
| **GPT-4**        | Provides ONLY the masculine form of the sentence.                                  |
| **GPT-4-turbo**  | DEPENDING on the target language, provides both variants or only the masculine one. |
| **GPT-4o-mini**  | DEPENDING on the target language, provides both variants or only the masculine one. |


# Experiment

## GPT-4o

### French

In [47]:
len(source_sentence) # check size

193

In [4]:
def experiment(model, language, sentence):
    '''
    Translates a given sentence into the specified target language using the specified model.

    Parameters:
    - model (str): The name of the model to use (e.g., 'gpt-4', 'gpt-4-turbo').
    - language (str): The target language for the translation.
    - sentence (str): The sentence to be translated.

    Returns:
    - str: The translated sentence generated by the model.
    '''
    
    response = client.chat.completions.create(
      model=model, #insert model
      messages=[
        {
          "role": "system",
          "content": "You are a helpful assistant"
        },
        {
          "role": "user",
          "content": f"Can you translate the following sentence into {language}: {sentence}"
        }
      ],
      temperature=0.2,
      max_tokens=100,
      top_p=1
    )
    # print(sentence, ': Done')
    return response.choices[0].message.content

In [49]:
df_exp = pd.DataFrame(columns=['eng', 'fra_m', 'fra_f', 'fra_n', 'output'])
df_exp['eng'] = df['sentence'].apply(lambda x: x.get('eng', {}).get('n'))
df_exp['fra_m'] = df['sentence'].apply(lambda x: x.get('fra', {}).get('m'))
df_exp['fra_f'] = df['sentence'].apply(lambda x: x.get('fra', {}).get('f'))
df_exp['fra_n'] = df['sentence'].apply(lambda x: x.get('fra', {}).get('n'))
df_exp

,eng,fra_m,fra_f,fra_n,output
0,I am Afghan.,Je suis Afghan.,Je suis Afghane.,None,NaN
1,I am Angolan.,Je suis Angolais.,Je suis Angolaise.,None,NaN
2,I am Albanian.,Je suis Albanais.,Je suis Albanaise.,None,NaN
3,I am Andorran.,Je suis Andorran.,Je suis Andorrane.,None,NaN
4,I am Emirati.,Je suis Emirien.,Je suis Emirienne.,None,NaN
...,...,...,...,...,...
188,I am Samoan.,Je suis Samoan.,Je suis Samoane.,None,NaN
189,I am Yemeni.,Je suis Yéménite.,Je suis Yéménite.,None,NaN
190,I am South African.,Je suis Sud-africain.,Je suis Sud-africaine.,None,NaN
191,I am Zambian.,Je suis Zambien.,Je suis Zambienne.,None,NaN


In [56]:
model = 'gpt-4o'
language = 'French'
df_exp['output'] = df_exp['eng'].apply(lambda sentence: experiment(model, language, sentence))

I am Afghan. : Done
I am Angolan. : Done
I am Albanian. : Done
I am Andorran. : Done
I am Emirati. : Done
I am Argentine. : Done
I am Armenian. : Done
I am Antiguan. : Done
I am Australian. : Done
I am Austrian. : Done
I am Azerbaijani. : Done
I am Burundian. : Done
I am Belgian. : Done
I am Beninese. : Done
I am Burkinabe. : Done
I am Bangladeshi. : Done
I am Bulgarian. : Done
I am Bahraini. : Done
I am Bahamian. : Done
I am Bosnian. : Done
I am Belarusian. : Done
I am Belizean. : Done
I am Bolivian. : Done
I am Brazilian. : Done
I am Barbadian. : Done
I am Bruneian. : Done
I am Bhutanese. : Done
I am Motswana. : Done
I am Central African. : Done
I am Canadian. : Done
I am Swiss. : Done
I am Chilean. : Done
I am Chinese. : Done
I am Ivorian. : Done
I am Cameroonian. : Done
I am Congolese. : Done
I am Congolese. : Done
I am Colombian. : Done
I am Comoran. : Done
I am Cape Verdian. : Done
I am Costa Rican. : Done
I am Cuban. : Done
I am Cypriot. : Done
I am Czech. : Done
I am German. : 

In [62]:
df_exp.to_csv('output/fra_4o_exp.csv', index=False)

In [65]:
df_exp = pd.read_csv('output/fra_4o_exp.csv')
df_exp

,eng,fra_m,fra_f,fra_n,output
0,I am Afghan.,Je suis Afghan.,Je suis Afghane.,NaN,"The translation of ""I am Afghan"" into French i..."
1,I am Angolan.,Je suis Angolais.,Je suis Angolaise.,NaN,"The translation of ""I am Angolan"" into French ..."
2,I am Albanian.,Je suis Albanais.,Je suis Albanaise.,NaN,"The translation of ""I am Albanian"" into French..."
3,I am Andorran.,Je suis Andorran.,Je suis Andorrane.,NaN,"The translation of ""I am Andorran"" into French..."
4,I am Emirati.,Je suis Emirien.,Je suis Emirienne.,NaN,"The translation of ""I am Emirati"" into French ..."
...,...,...,...,...,...
188,I am Samoan.,Je suis Samoan.,Je suis Samoane.,NaN,"The translation of ""I am Samoan"" into French i..."
189,I am Yemeni.,Je suis Yéménite.,Je suis Yéménite.,NaN,"The translation of ""I am Yemeni"" into French i..."
190,I am South African.,Je suis Sud-africain.,Je suis Sud-africaine.,NaN,"The sentence ""I am South African"" can be trans..."
191,I am Zambian.,Je suis Zambien.,Je suis Zambienne.,NaN,"The translation of ""I am Zambian"" into French ..."


### Spanish

In [67]:
df_exp = pd.DataFrame(columns=['eng', 'spa_m', 'spa_f', 'spa_n', 'output'])
df_exp['eng'] = df['sentence'].apply(lambda x: x.get('eng', {}).get('n'))
df_exp['spa_m'] = df['sentence'].apply(lambda x: x.get('spa', {}).get('m'))
df_exp['spa_f'] = df['sentence'].apply(lambda x: x.get('spa', {}).get('f'))
df_exp['spa_n'] = df['sentence'].apply(lambda x: x.get('spa', {}).get('n'))
df_exp

model = 'gpt-4o'
language = 'Spanish'
df_exp['output'] = df_exp['eng'].apply(lambda sentence: experiment(model, language, sentence))

df_exp.to_csv('output/spa_4o_exp.csv', index=False)

I am Afghan. : Done
I am Angolan. : Done
I am Albanian. : Done
I am Andorran. : Done
I am Emirati. : Done
I am Argentine. : Done
I am Armenian. : Done
I am Antiguan. : Done
I am Australian. : Done
I am Austrian. : Done
I am Azerbaijani. : Done
I am Burundian. : Done
I am Belgian. : Done
I am Beninese. : Done
I am Burkinabe. : Done
I am Bangladeshi. : Done
I am Bulgarian. : Done
I am Bahraini. : Done
I am Bahamian. : Done
I am Bosnian. : Done
I am Belarusian. : Done
I am Belizean. : Done
I am Bolivian. : Done
I am Brazilian. : Done
I am Barbadian. : Done
I am Bruneian. : Done
I am Bhutanese. : Done
I am Motswana. : Done
I am Central African. : Done
I am Canadian. : Done
I am Swiss. : Done
I am Chilean. : Done
I am Chinese. : Done
I am Ivorian. : Done
I am Cameroonian. : Done
I am Congolese. : Done
I am Congolese. : Done
I am Colombian. : Done
I am Comoran. : Done
I am Cape Verdian. : Done
I am Costa Rican. : Done
I am Cuban. : Done
I am Cypriot. : Done
I am Czech. : Done
I am German. : 

### German

In [68]:
df_exp = pd.DataFrame(columns=['eng', 'deu_m', 'deu_f', 'deu_n', 'output'])
df_exp['eng'] = df['sentence'].apply(lambda x: x.get('eng', {}).get('n'))
df_exp['deu_m'] = df['sentence'].apply(lambda x: x.get('deu', {}).get('m'))
df_exp['deu_f'] = df['sentence'].apply(lambda x: x.get('deu', {}).get('f'))
df_exp['deu_n'] = df['sentence'].apply(lambda x: x.get('deu', {}).get('n'))
df_exp

model = 'gpt-4o'
language = 'German'
df_exp['output'] = df_exp['eng'].apply(lambda sentence: experiment(model, language, sentence))

df_exp.to_csv('output/deu_4o_exp.csv', index=False)

I am Afghan. : Done
I am Angolan. : Done
I am Albanian. : Done
I am Andorran. : Done
I am Emirati. : Done
I am Argentine. : Done
I am Armenian. : Done
I am Antiguan. : Done
I am Australian. : Done
I am Austrian. : Done
I am Azerbaijani. : Done
I am Burundian. : Done
I am Belgian. : Done
I am Beninese. : Done
I am Burkinabe. : Done
I am Bangladeshi. : Done
I am Bulgarian. : Done
I am Bahraini. : Done
I am Bahamian. : Done
I am Bosnian. : Done
I am Belarusian. : Done
I am Belizean. : Done
I am Bolivian. : Done
I am Brazilian. : Done
I am Barbadian. : Done
I am Bruneian. : Done
I am Bhutanese. : Done
I am Motswana. : Done
I am Central African. : Done
I am Canadian. : Done
I am Swiss. : Done
I am Chilean. : Done
I am Chinese. : Done
I am Ivorian. : Done
I am Cameroonian. : Done
I am Congolese. : Done
I am Congolese. : Done
I am Colombian. : Done
I am Comoran. : Done
I am Cape Verdian. : Done
I am Costa Rican. : Done
I am Cuban. : Done
I am Cypriot. : Done
I am Czech. : Done
I am German. : 

### Italian

In [75]:
df['sentence'] = df['sentence'].apply(lambda x: {**x, 'it': ast.literal_eval(x['it'])} if isinstance(x['it'], str) else x) # there were problem with it dict

df_exp = pd.DataFrame(columns=['eng', 'it_m', 'it_f', 'it_n', 'output'])

# Extracting Italian translations instead of German
df_exp['eng'] = df['sentence'].apply(lambda x: x.get('eng', {}).get('n'))
df_exp['it_m'] = df['sentence'].apply(lambda x: x.get('it', {}).get('m'))
df_exp['it_f'] = df['sentence'].apply(lambda x: x.get('it', {}).get('f'))
df_exp['it_n'] = df['sentence'].apply(lambda x: x.get('it', {}).get('n'))

# Generate translations using the model
model = 'gpt-4o'
language = 'Italian'
df_exp['output'] = df_exp['eng'].apply(lambda sentence: experiment(model, language, sentence))

# Save the DataFrame to a CSV file
df_exp.to_csv('output/it_4o_exp.csv', index=False)

I am Afghan. : Done
I am Angolan. : Done
I am Albanian. : Done
I am Andorran. : Done
I am Emirati. : Done
I am Argentine. : Done
I am Armenian. : Done
I am Antiguan. : Done
I am Australian. : Done
I am Austrian. : Done
I am Azerbaijani. : Done
I am Burundian. : Done
I am Belgian. : Done
I am Beninese. : Done
I am Burkinabe. : Done
I am Bangladeshi. : Done
I am Bulgarian. : Done
I am Bahraini. : Done
I am Bahamian. : Done
I am Bosnian. : Done
I am Belarusian. : Done
I am Belizean. : Done
I am Bolivian. : Done
I am Brazilian. : Done
I am Barbadian. : Done
I am Bruneian. : Done
I am Bhutanese. : Done
I am Motswana. : Done
I am Central African. : Done
I am Canadian. : Done
I am Swiss. : Done
I am Chilean. : Done
I am Chinese. : Done
I am Ivorian. : Done
I am Cameroonian. : Done
I am Congolese. : Done
I am Congolese. : Done
I am Colombian. : Done
I am Comoran. : Done
I am Cape Verdian. : Done
I am Costa Rican. : Done
I am Cuban. : Done
I am Cypriot. : Done
I am Czech. : Done
I am German. : 

## GPT 3.5-turbo

In [5]:
from tqdm.auto import tqdm
tqdm.pandas(desc="Translation")

### French

In [81]:
df_exp = pd.DataFrame(columns=['eng', 'fra_m', 'fra_f', 'fra_n', 'output'])
df_exp['eng'] = df['sentence'].apply(lambda x: x.get('eng', {}).get('n'))
df_exp['fra_m'] = df['sentence'].apply(lambda x: x.get('fra', {}).get('m'))
df_exp['fra_f'] = df['sentence'].apply(lambda x: x.get('fra', {}).get('f'))
df_exp['fra_n'] = df['sentence'].apply(lambda x: x.get('fra', {}).get('n'))

model = 'gpt-3.5-turbo'
language = 'French'
df_exp['output'] = df_exp['eng'].progress_apply(lambda sentence: experiment(model, language, sentence))

df_exp.to_csv('output/fra_3.5-turbo_exp.csv', index=False)

Translation!:   0%|          | 0/193 [00:00<?, ?it/s]

### Spanish

In [83]:
df_exp = pd.DataFrame(columns=['eng', 'spa_m', 'spa_f', 'spa_n', 'output'])
df_exp['eng'] = df['sentence'].apply(lambda x: x.get('eng', {}).get('n'))
df_exp['spa_m'] = df['sentence'].apply(lambda x: x.get('spa', {}).get('m'))
df_exp['spa_f'] = df['sentence'].apply(lambda x: x.get('spa', {}).get('f'))
df_exp['spa_n'] = df['sentence'].apply(lambda x: x.get('spa', {}).get('n'))

model = 'gpt-3.5-turbo'
language = 'Spanish'
df_exp['output'] = df_exp['eng'].progress_apply(lambda sentence: experiment(model, language, sentence))

df_exp.to_csv('output/spa_3.5-turbo_exp.csv', index=False)

Translation:   0%|          | 0/193 [00:00<?, ?it/s]

### German

In [84]:
df_exp = pd.DataFrame(columns=['eng', 'deu_m', 'deu_f', 'deu_n', 'output'])
df_exp['eng'] = df['sentence'].apply(lambda x: x.get('eng', {}).get('n'))
df_exp['deu_m'] = df['sentence'].apply(lambda x: x.get('deu', {}).get('m'))
df_exp['deu_f'] = df['sentence'].apply(lambda x: x.get('deu', {}).get('f'))
df_exp['deu_n'] = df['sentence'].apply(lambda x: x.get('deu', {}).get('n'))

model = 'gpt-3.5-turbo'
language = 'German'
df_exp['output'] = df_exp['eng'].progress_apply(lambda sentence: experiment(model, language, sentence))

df_exp.to_csv('output/deu_3.5-turbo_exp.csv', index=False)

Translation:   0%|          | 0/193 [00:00<?, ?it/s]

### Italian

In [85]:
df_exp = pd.DataFrame(columns=['eng', 'it_m', 'it_f', 'it_n', 'output'])
df_exp['eng'] = df['sentence'].apply(lambda x: x.get('eng', {}).get('n'))
df_exp['it_m'] = df['sentence'].apply(lambda x: x.get('it', {}).get('m'))
df_exp['it_f'] = df['sentence'].apply(lambda x: x.get('it', {}).get('f'))
df_exp['it_n'] = df['sentence'].apply(lambda x: x.get('it', {}).get('n'))

model = 'gpt-3.5-turbo'
language = 'Italian'
df_exp['output'] = df_exp['eng'].progress_apply(lambda sentence: experiment(model, language, sentence))

df_exp.to_csv('output/it_3.5-turbo_exp.csv', index=False)

Translation:   0%|          | 0/193 [00:00<?, ?it/s]

## GPT-4/4-turbo/4o-mini

In [10]:
import pandas as pd
from tqdm import tqdm

# Add progress bar functionality
tqdm.pandas()

def process_sentences(df, languages, models, output_folder):
    """
    Processes sentences for multiple languages and models.

    Args:
        df (pd.DataFrame): The input DataFrame containing sentences.
        languages (dict): A dictionary where keys are language codes (e.g., 'fra', 'spa') 
                          and values are full language names (e.g., 'French', 'Spanish').
        models (list): A list of model names to use for processing.
        output_folder (str): The folder to save output CSV files.

    Returns:
        None
    """
    for model in models:
        for lang_code, lang_name in languages.items():
            print(model, lang_name, sep='-')
            # Create a new DataFrame for the current language
            df_exp = pd.DataFrame(columns=['eng', f'{lang_code}_m', f'{lang_code}_f', f'{lang_code}_n', 'output'])
            df_exp['eng'] = df['sentence'].apply(lambda x: x.get('eng', {}).get('n'))
            df_exp[f'{lang_code}_m'] = df['sentence'].apply(lambda x: x.get(lang_code, {}).get('m'))
            df_exp[f'{lang_code}_f'] = df['sentence'].apply(lambda x: x.get(lang_code, {}).get('f'))
            df_exp[f'{lang_code}_n'] = df['sentence'].apply(lambda x: x.get(lang_code, {}).get('n'))

            # Apply the experiment function
            df_exp['output'] = df_exp['eng'].progress_apply(lambda sentence: experiment(model, lang_name, sentence))

            # Save the results to a CSV file
            output_path = f"{output_folder}/{lang_code}_{model.split('gpt-')[1]}_exp.csv"
            df_exp.to_csv(output_path, index=False)
            print(f"Saved results to {output_path}")

# Example Usage
languages = {
    # 'fra': 'French',
    # 'spa': 'Spanish',
    # 'deu': 'German'
    'it': 'Italian'
}

models = ['gpt-4', 'gpt-4-turbo', 'gpt-4o-mini']

In [11]:
process_sentences(df, languages, models, output_folder='output')

gpt-4-Italian


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████| 193/193 [03:31<00:00,  1.09s/it]


Saved results to output/it_4_exp.csv
gpt-4-turbo-Italian


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████| 193/193 [06:02<00:00,  1.88s/it]


Saved results to output/it_4-turbo_exp.csv
gpt-4o-mini-Italian


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████| 193/193 [04:43<00:00,  1.47s/it]

Saved results to output/it_4o-mini_exp.csv


In [9]:
process_sentences(df, languages, models, output_folder='output')

gpt-4-French


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████| 193/193 [03:03<00:00,  1.05it/s]


Saved results to output/fra_4_exp.csv
gpt-4-Spanish


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████| 193/193 [03:13<00:00,  1.00s/it]


Saved results to output/spa_4_exp.csv
gpt-4-German


...

Saved results to output/fra_4o-mini_exp.csv
gpt-4o-mini-Spanish


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████| 193/193 [04:36<00:00,  1.43s/it]


Saved results to output/spa_4o-mini_exp.csv
gpt-4o-mini-German


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████| 193/193 [05:20<00:00,  1.66s/it]

Saved results to output/deu_4o-mini_exp.csv
